In [1]:
import os

In [2]:
%pwd

'/Users/user/Desktop/Projects/end-to-end-ds/research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.end_to_end_ds.constants import *
from src.end_to_end_ds.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEME_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import urllib.request as request
import zipfile
from src.end_to_end_ds import logger

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: {headers}")
        
        else:
            logger.info("File already exist")
        
    def extract_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file()
except Exception as e:
    raise e

[2025-08-08 17:55:28,811: INFO: common: Yaml file config/config.yaml loaded successfully]
[2025-08-08 17:55:28,813: INFO: common: Yaml file params.yaml loaded successfully]
[2025-08-08 17:55:28,814: INFO: common: Yaml file schema.yaml loaded successfully]
[2025-08-08 17:55:28,815: INFO: common: created directory at artifacts]
[2025-08-08 17:55:28,816: INFO: common: created directory at artifacts/data_ingestion]
[2025-08-08 17:55:28,813: INFO: common: Yaml file params.yaml loaded successfully]
[2025-08-08 17:55:28,814: INFO: common: Yaml file schema.yaml loaded successfully]
[2025-08-08 17:55:28,815: INFO: common: created directory at artifacts]
[2025-08-08 17:55:28,816: INFO: common: created directory at artifacts/data_ingestion]
[2025-08-08 17:55:29,761: INFO: 2713055599: artifacts/data_ingestion/data.zip download with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Conte